<a href="https://colab.research.google.com/github/alexrzntsv/TOBD-FU/blob/main/8.%20profiling/08_profiling_%D0%A0%D1%8F%D0%B7%D0%B0%D0%BD%D1%86%D0%B5%D0%B2_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B5%D0%B9_%D0%B22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [19]:
import numpy as np
import numba

Инициализация массива A:

In [2]:
A = np.random.randint(0,1000, size=(1000000))
A

array([535, 719, 461, ..., 107, 950, 823])

Варианты решения:

In [3]:
def mean_B(A):
  sum = 0
  for a in A:
    b = a + 100
    sum += b
  return sum/len(A)

%timeit mean_B(A)

293 ms ± 9.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
def mean_B(A):
  B = A + 100
  return B.mean()

%timeit mean_B(A)

2.76 ms ± 72.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
@numba.njit
def mean_B(A):
    sum = 0
    for a in A:
      b = a + 100
      sum += b
    return sum/len(A)

%timeit mean_B(A)

638 µs ± 171 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
@numba.njit
def mean_B(A):
  B = A + 100
  return B.mean()

%timeit mean_B(A)

3.71 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [2]:
import numpy as np
import pandas as pd

In [10]:
df = pd.DataFrame(np.random.randint(0, 1000, size=(2000000, 4)),
                  columns=['1', '2', '3', '4'])

A_eng = ord('A')
eng = [chr(i) for i in range(A_eng, A_eng + 26)]


df['key'] = np.random.choice(eng, 2000000)

In [11]:
df.head()

,1,2,3,4,key
0,857,366,928,530,A
1,423,396,354,763,U
2,602,20,709,980,B
3,388,860,882,94,Z
4,360,113,214,504,S


In [13]:
def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

%timeit g(df)    

819 ms ± 90.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
def g(df):
    return df[df["key"].str.contains("a|b|c|d|e")]

%timeit g(df)

661 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
def g(df):
    return df[df["key"].isin(('a', 'b', 'c', 'd', 'e', 'f', 'g'))]

%timeit g(df)

77.4 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Лабораторная работа 3

In [1]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 673 kB 5.0 MB/s 


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [4]:
recipes = pd.read_csv('recipes_sample.csv', index_col='id')
recipes.head(2)

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
id,,,,,,,
44123,george s at the cove black bean soup,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
67664,healthy for them yogurt popsicles,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN


In [6]:
reviews = pd.read_csv('reviews_sample.csv', index_col=0)
reviews = reviews.astype({'date': 'datetime64[ns]'})
reviews.head(2)


,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...


In [7]:
def A(df):
  s = 0
  num_rows = 0
  for index, row in df.iterrows():
    if row["date"].year == 2010:
      s += row["rating"]
      num_rows += 1

  return s/num_rows

%timeit A(reviews)

5.64 s ± 36.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
def B(df):
  df = df.loc[df['date'].dt.year == 2010]
  s = 0
  for index, row in df.iterrows():
      s += row["rating"]

  return s/df.shape[0]

%timeit b_mean = B(reviews)

551 ms ± 6.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
def C(df):
  df = df.loc[df['date'].dt.year == 2010]
  return df['rating'].mean()

%timeit C(reviews)

18.4 ms ± 75.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
a_mean = A(reviews)
b_mean = B(reviews)
c_mean = C(reviews)

assert a_mean == b_mean and b_mean == c_mean

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [11]:
%load_ext line_profiler

In [12]:
%lprun -f A A(reviews)

Timer unit: 1e-09 s

Total time: 11.1637 s
File: <ipython-input-77-38d409270a75>
Function: A at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def A(df):
     2         1       1491.0   1491.0      0.0    s = 0
     3         1        283.0    283.0      0.0    num_rows = 0
     4    126696 9924610661.0  78334.0     88.9    for index, row in df.iterrows():
     5    114602 1139830730.0   9946.0     10.2      if row["date"].year == 2010:
     6     12094   95049155.0   7859.2      0.9        s += row["rating"]
     7     12094    4164654.0    344.4      0.0        num_rows += 1
     8                                           
     9         1       1300.0   1300.0      0.0    return s/num_rows

In [13]:
%lprun -f B B(reviews)

Timer unit: 1e-09 s

Total time: 1.03476 s
File: <ipython-input-61-c946e4d7383a>
Function: B at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def B(df):
     2         1   28731860.0 28731860.0      2.8    df = df.loc[df['date'].dt.year == 2010]
     3         1        373.0    373.0      0.0    s = 0
     4     12094  891263528.0  73694.7     86.1    for index, row in df.iterrows():
     5     12094  114757495.0   9488.8     11.1        s += row["rating"]
     6                                           
     7         1      11209.0  11209.0      0.0    return s/df.shape[0]

In [14]:
%lprun -f C C(reviews)

Timer unit: 1e-09 s

Total time: 0.0216657 s
File: <ipython-input-74-f25600a60908>
Function: C at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def C(df):
     2         1   21257260.0 21257260.0     98.1    df = df.loc[df['date'].dt.year == 2010]
     3         1     408486.0 408486.0      1.9    return df['rating'].mean()

Последний вариант функции работает наиболее быстро. Благодаря профайлеру видно, что главный недостаток первых двух функций - цикл. А у первой ещё и сравнение в цикле.

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [15]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [16]:
%timeit get_word_reviews_count(reviews)

18.7 s ± 368 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-09 s

Total time: 40.5759 s
File: <ipython-input-85-b1bc049bcd0c>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1       1532.0   1532.0      0.0      word_reviews = {}
     3    126679 10853789689.0  85679.5     26.7      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679 2179751980.0  17206.9      5.4          recipe_id, review = row['recipe_id'], row['review']
     5    126679  594752800.0   4695.0      1.5          words = review.split(' ')
     6   6792010 1265133644.0    186.3      3.1          for word in words:
     7   6617066 2857892925.0    431.9      7.0              if word not in word_reviews:
     8    174944   80934833.0    462.6      0.2                  word_reviews[word] = []
     9   6792010 3540894384.0    521.3      8.7              word_reviews[word].append(recipe_id)
    10                                               
    11         1        871.0    871.0      0.0      word_reviews_count = {}
    12    126679 10599450102.0  83671.7     26.1      for _, row in df.dropna(subset=['review']).iterrows():
    13    126679 1330550693.0  10503.3      3.3          review = row['review']
    14    126679  593146826.0   4682.3      1.5          words = review.split(' ')
    15   6792010 1360529434.0    200.3      3.4          for word in words:
    16   6792010 5319086062.0    783.1     13.1              word_reviews_count[word] = len(word_reviews[word])
    17         1        435.0    435.0      0.0      return word_reviews_count

Узкое место программы - циклы (их зачем-то 2, хотя можно 1). В идеале - циклы нужно убрать. Также много времени тратится на запоминание всех id во вложенном списке word_reviews и его последующем использовании во втором цикле.

In [18]:
def get_word_reviews_count_2(df):
  d = df.review.str.split().explode().value_counts()
  return d.to_dict()

In [19]:
%timeit get_word_reviews_count_2(reviews)

2.07 s ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%lprun -f get_word_reviews_count_2 get_word_reviews_count_2(reviews)

Timer unit: 1e-09 s

Total time: 4.0439 s
File: <ipython-input-100-acab8ba60f91>
Function: get_word_reviews_count_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count_2(df):
     2         1 3784552905.0 3784552905.0     93.6    d = df.review.str.split().explode().value_counts()
     3         1  259347733.0 259347733.0      6.4    return d.to_dict()

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [21]:
reviews = reviews.drop(reviews[reviews['rating'] == 0].index)
reviews.rating.value_counts()

5    91361
4    20866
3     4635
2     1628
1     1401
Name: rating, dtype: int64

In [22]:
reviews['mean'] = reviews.groupby('recipe_id').rating.transform('mean')

In [23]:
reviews.head(2)

,user_id,recipe_id,date,rating,review,mean
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,4.818182
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,5.000000


In [25]:
def mape_1(df):
  s = 0
  num_rows = 0
  for index, row in df.iterrows():
      s += abs(row["rating"] - row['mean']) / row['rating']
      num_rows += 1
  return s/num_rows

%timeit mape_1(reviews)

6.44 s ± 294 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
import warnings
warnings.filterwarnings("ignore")

In [46]:
rating = reviews['rating'].to_list()
mean = reviews['mean'].to_list() 

@numba.njit
def mape_2(rating, mean):
  s = 0
  num_rows = 0
  for i in range(len(rating)):
      s += abs(rating[i] - mean[i]) / rating[i]
      num_rows += 1
  return s/num_rows

%timeit mape_2(rating, mean)

352 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
def mape_3(df):
  rating = df['rating'].to_numpy()
  mean = df['mean'].to_numpy() 
  mape = abs(rating - mean) / rating
  return mape.mean()

%timeit mape_3(reviews)

788 µs ± 14.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [47]:
rating = reviews['rating'].to_numpy()
mean = reviews['mean'].to_numpy() 

@numba.jit
def mape_4(rating, mean):
  mape = abs(rating - mean) / rating
  return mape.mean()

%timeit mape_4(rating, mean)

837 µs ± 9.54 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
